In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from matplotlib import font_manager, rc
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')
import platform
plt.rc('font', family='NanumBarunGothic') 

from sklearn.ensemble import RandomForestRegressor

import matplotlib.font_manager as fm
#fm._rebuild()
plt.rc('font', family='NanumBarunGothic') 

import lightgbm as lgb

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('display.max_columns', None)

train = pd.read_csv('/gdrive/My Drive/train_new.csv')
test = pd.read_csv('/gdrive/My Drive/test_new.csv')
ratings = pd.read_csv('/gdrive/My Drive/tv_new.csv')
category = pd.read_csv('/gdrive/My Drive/categorized_HJ2.csv') # 카테고리 추가되었습니다.

In [ ]:
# train셋의 취급액 이름이 달라서 통일시켜줍니다. 우리의 소원은 통일.
train = train.rename(columns = {' 취급액 ':'취급액'})

In [ ]:
# 전처리를 한 번에 해주기 위해서 train, test를 합쳐줍니다.
df_all = pd.concat([train, test]).reset_index()

In [ ]:
def make_cast_time(train): # nan값이 있는 초기 데이터셋을 써야 하기 때문에 가장 먼저 돌려줍니다. 자세한 알고리즘은 창고에서 찾아보세요.

    # 일단 nan값을 노출칼럼에만 남기기 위해서 나머지 nan값 전처리 미리 해버리겠습니다.
    train['취급액'] = train['취급액'].fillna('0')
    train = train[train.상품군 != '무형']

    # nan값 있는 애들을 따로 저장해둡니다.
    train_yes_nan = train[train['노출(분)'].isnull()]
    # nan값 없는 애들을 따로 저장해줍니다. 이놈들로 작업할겁니다.
    train_not_nan = train.dropna()

    # 인덱스 꼬여서 포문 안 돌아가기 때문에 인덱스 리셋하고 나중에 불러와주기 위해서 따로 저장해줍니다.
    train_not_nan_index = train_not_nan.index
    # 리셋 인덱스 해줍니다.
    train_not_nan_reset_index = train_not_nan.reset_index(drop=True)

    # 여기서부터 cast_time, cast_count 포문-----------------------------------------------------
    cast_time = []
    cast_count = []
    time = 0.
    count = 1

    for i in range(len(train_not_nan_reset_index)):
        if i == 0: # 첫번째 행은 이걸로 하십쇼.
            time = train_not_nan_reset_index['노출(분)'][i] # 더하지 않고, 자기 노출을 그대로 가져옵니다.
            count = 1 # count도 1로 초기화 합니다.
            cast_time.append(time) # 그리고 어펜드.
            cast_count.append(count) # 그리고 어펜드.
        elif train_not_nan_reset_index['상품코드'][i] == train_not_nan_reset_index['상품코드'][i-1]: # 자기의 상품코드가 앞행의 상품코드와 같을 때
            time += train_not_nan_reset_index['노출(분)'][i] # 기존 누적된 노출에 자기 노출을 더한다.
            count += 1 # count도 1을 더해줍니다.
            cast_time.append(time) # 그리고 어펜드.
            cast_count.append(count) # 그리고 어펜드.
        else: # 근데 자기 상품코드와 앞행의 상품코드가 다르다면, 새로운 방송이 시작된 거겠죠?
            time = train_not_nan_reset_index['노출(분)'][i] # 더하지 않고, 자기 노출을 그대로 가져옵니다.
            count = 1 # count도 1로 초기화 합니다.
            cast_time.append(time) # 그리고 어펜드.
            cast_count.append(count) # 그리고 어펜드.
    # 포문 끝------------------------------------------------------------  
    # 컬럼으로 박아줍니다.
    train_not_nan_reset_index['cast_time'] = cast_time
    train_not_nan_reset_index['cast_count'] = cast_count
    
    # 여기서부터 cast_time_sum, cast_count_sum 포문-------------------------------------------------
    cast_time_sum = []
    cast_count_sum = []

    for i in range(len(train_not_nan_reset_index)):
        if i == max(train_not_nan_reset_index.index): #마지막은 비교할 다음타자가 없으니 바로 넣기.
            cast_time_sum.append(train_not_nan_reset_index['cast_time'][i])
            cast_count_sum.append(train_not_nan_reset_index['cast_count'][i])
        elif train_not_nan_reset_index['상품코드'][i] == train_not_nan_reset_index['상품코드'][i+1]: # 자기와 다음타자의 상품코드가 같으면
            cast_time_sum.append(np.nan)
            cast_count_sum.append(np.nan)
        else: # 자기와 다음타자의 상품코드가 다르면
            cast_time_sum.append(train_not_nan_reset_index['cast_time'][i])
            cast_count_sum.append(train_not_nan_reset_index['cast_count'][i])
            
    # 포문 끝----------------------------------------------------------------------
    # 칼럼으로 박아줍니다.
    train_not_nan_reset_index['cast_time_sum'] = cast_time_sum
    train_not_nan_reset_index['cast_count_sum'] = cast_count_sum
    # 백필로 nan값 채워줍니다.
    train_not_nan_reset_index.fillna(method='bfill', inplace=True)
    

    # 원래 인덱스 다시 넣어줍니다.
    train_not_nan_reset_index.index = train_not_nan_index
    # 컨캣으로 밑으로 붙여주고,
    train_concat = pd.concat([train_not_nan_reset_index, train_yes_nan], axis=0)
    # 소트 인덱스 먼저 하고,
    train_concat_sort_index = train_concat.sort_index()
    # ffill 로 채워줍니다. 끝. 이거 너무 힘들었다 진짜 레알.
    train = train_concat_sort_index.fillna(method='ffill')
    
    # 누적 방송 시간 비율 칼럼을 만들어줍니다.
    train['cast_time_ratio'] = train.cast_time / train.cast_time_sum
    
    # 인덱스 리셋해줍니다.
    train = train.reset_index(drop=True)
    
    # 다이어트
    train['cast_count'] = train.cast_count.astype(np.int16)
    
    return train

In [ ]:
df_all = make_cast_time(df_all)

In [ ]:
# 기본 전처리 함수
def preprocessing_1(df):
    
    # 취급액과 판매단가 수치형 변수로 바꾸기.
    df['취급액'] = df['취급액'].apply(lambda x: x.replace(',', ''))
    df['판매단가'] = df['판매단가'].apply(lambda x: x.replace(',', ''))
    df['취급액'] = df['취급액'].astype(np.int32)
    df['판매단가'] = df['판매단가'].astype(np.int32)
    
    # 칼럼명 너무 어려우니까 쪼금만 바꿔줍니다.
    #df = df.rename(columns = {' 취급액 ':'취급액'})
    
    return df

In [ ]:
df_all = preprocessing_1(df_all)

In [ ]:
# 상품명 전처리 함수
def preprocessing_3(train):
    # 1. 일시불/무이자/없음
    train.loc[train['상품명'].str.contains('일시불') == True, "상품명_plan"] = '1'
    train.loc[train['상품명'].str.contains('일\)') == True, "상품명_plan"] = '1'
    train.loc[train['상품명'].str.contains('무이자') == True, "상품명_plan"] = '2'
    train.loc[train['상품명'].str.contains('무\)') == True, "상품명_plan"] = '2'
    train['상품명_plan'] = train['상품명_plan'].fillna('0')

    # 2. 추가구성/단품구성
    train.loc[train['상품명'].str.contains('\+') == True, "상품명_add"] = '1'
    train['상품명_add'] = train['상품명_add'].fillna('0')

    # 3. 기타/삼성/LG
    train.loc[train['상품명'].str.contains('삼성') == True, "상품명_maker"] = '1'
    train.loc[train['상품명'].str.contains('LG') == True, "상품명_maker"] = '2'
    train['상품명_maker'] = train['상품명_maker'].fillna('0')

    # 4. 세트구성/단품구성
    train.loc[train['상품명'].str.contains('세트') == True, "상품명_set"] = '1'
    train['상품명_set'] = train['상품명_set'].fillna('0')

    # 5. 여성/남성/없음
    train.loc[train['상품명'].str.contains('여성') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('브라') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('란쥬') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('블라우스') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('밍크') == True, "상품명_sex"] = '1'
    train.loc[train['상품명'].str.contains('남성') == True, "상품명_sex"] = '2'
    train.loc[train['상품명'].str.contains('드로즈') == True, "상품명_sex"] = '2'
    train.loc[train['상품명'].str.contains('트렁크') == True, "상품명_sex"] = '2'
    train['상품명_sex'] = train['상품명_sex'].fillna('0')
    
    # 6. 아동/성인
    train.loc[train['상품명'].str.contains('주니어') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('여아') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('남아') == True, "상품명_kid"] = '1'
    train.loc[train['상품명'].str.contains('아동') == True, "상품명_kid"] = '1'
    train['상품명_kid'] = train['상품명_kid'].fillna('0')

    # category로 형변환해서 다이어트 시켜줍니다.
    train['상품명_plan'] = train['상품명_plan'].astype('category')
    train['상품명_add'] = train['상품명_add'].astype('category')
    train['상품명_maker'] = train['상품명_maker'].astype('category')
    train['상품명_set'] = train['상품명_set'].astype('category')
    train['상품명_sex'] = train['상품명_sex'].astype('category')
    
    return train

In [ ]:
df_all = preprocessing_3(df_all)

In [ ]:
def make_fake_weight(train):
    fake_weight = []
    weight = 1

    for i in range(len(train)):
        if i == 0: # 첫번째 행은 이걸로 하십쇼.
            weight = 1 # count도 1로 초기화 합니다.
            fake_weight.append(weight) # 그리고 어펜드.
            
        elif train['마더코드'][i] == train['마더코드'][i-1]: # 자기의 마더코드가 앞행의 것과 같을 때
            if train['판매단가'][i] < train['판매단가'][i-1]: # 자기의 단가가 앞행의 것보다 작으면(같으면 안됨)
                weight +=1 # 가중치 1을 더하고,
                fake_weight.append(weight) # 그리고 어펜드
            else: # 자기의 단가가 앞행의 것과 같거나 더 크면(더 클 수는 없음.)
                weight = 1 # 가중치 1로 초기화
                fake_weight.append(weight)
                
        else: # 근데 자기 마더코드와 앞행의 마더코드가 다르다면, 새로운 방송이 시작된 거겠죠?
            weight = 1 # 가중치 1로 초기화
            fake_weight.append(weight)
            
    train['fake_weight'] = fake_weight
    train['fake_weight'] = train.fake_weight.apply(lambda x: -(x*x))
    
    return train

In [ ]:
df_all = make_fake_weight(df_all)

In [ ]:
def make_train_fake(train):
    
    # 더 정확한 변수 생성을 위해 기계가 걸러내지 못하는 조건을 쳐내줍니다.
    train_fake = train[train.상품명_plan == '0'] # 할부플랜이 다른 애들 쳐내기
    train_fake = train_fake[train_fake.상품명_sex == '0'] # 성별이 다른 애들 쳐내기.
    
    # 포문 돌리기 위해 인덱스 리셋 해줍니다.
    train_fake.reset_index(drop=True, inplace=True)

    return train_fake

In [ ]:
def make_fake_weight2(train):
    fake_weight = []
    weight = 0

    for i in range(len(train)):
        if i == 0: # 첫번째 행은 이걸로 하십쇼.
            weight = 0 # count도 1로 초기화 합니다.
            fake_weight.append(weight) # 그리고 어펜드.
            
        elif train['마더코드'][i] == train['마더코드'][i-1]: # 자기의 마더코드가 앞행의 것과 같을 때
            if train['판매단가'][i] == train['판매단가'][i-1]: # 자기의 단가가 앞행의 것과 같을 때
                if train['방송일시'][i] == train['방송일시'][i-1]:
                    if train['상품코드'][i] != train['상품코드'][i-1]: # 상품코드는 일치하지 않으면 조건 성립
                        weight -=1 # 가중치 1을 빼고,
                        fake_weight.append(weight) # 그리고 어펜드
                    else: # 상품코드까지 같으면 동일 상품.
                        weight = 0
                        fake_weight.append(weight)
                else: #방송일시가 다르면 새로운 방송 시작
                    weight = 0 # 가중치 0로 초기화
                    fake_weight.append(weight)
            else: # 판매단가가 다르면 다른 상품
                weight = 0 # 가중치 0로 초기화
                fake_weight.append(weight)
        else: # 근데 자기 마더코드와 앞행의 마더코드가 다르다면, 새로운 방송이 시작된 거겠죠?
            weight = 0 # 가중치 1로 초기화
            fake_weight.append(weight)
            
    train['fake_weight2'] = fake_weight
    
    return train

In [ ]:
df_fake = make_train_fake(df_all)
df_fake = make_fake_weight2(df_fake)

In [ ]:
# 기존 데이터에 합쳐줍니다.
df_all = pd.merge(df_all, df_fake[['방송일시','마더코드','상품코드','상품명','판매단가', 'fake_weight2']],
                 on=['방송일시','마더코드','상품코드','상품명','판매단가'], how='left')

# 빈 값은 전부 0으로 채워줍니다.
df_all['fake_weight2'] = df_all['fake_weight2'].fillna(0)

In [ ]:
category = pd.read_csv('/gdrive/My Drive/categorized_HJ2.csv') # 카테고리 추가되었습니다.

In [ ]:
# 난 값 먼저 채워 줍니다.
category['cat3'] = category.apply(lambda x: x['cat2'] if x['cat3'] != x['cat3'] else x['cat3'], axis=1)

In [ ]:
def preprocessing_to_merge(df):
    # merge를 위해 datetime으로 바꿔줍니다..
    df['방송일시'] = pd.to_datetime(df['방송일시'], errors='coerce')
    df['방송일시'] = df['방송일시'].astype(str)
    
    # key 칼럼을 만들어줍니다. 방송일시만으로는 안되니, 방송일시+상품명을 해줍니다.
    df['key'] = df['방송일시'] + df['상품명']
    
    return df

In [ ]:
def preprocessing_merging(df1, df2):
    # df1이 왼쪽, df2가 오른쪽입니다.
    df3 = pd.merge(df1, df2[['key','ratings_mean','cat1','cat2','cat3']], how='left', on=['key'])
    df3 = df3.drop('key', axis=1) # key 칼럼은 드랍.
    
    return df3

In [ ]:
df_all = preprocessing_to_merge(df_all)
category = preprocessing_to_merge(category)
df_all = preprocessing_merging(df_all, category)

In [ ]:
# 방송일시 쪼개기 전처리 함수
def preprocessing_4(train):
    # datetime으로 바꿔줍니다.
    train['방송일시'] = pd.to_datetime(train['방송일시'], errors='coerce')
   
    # 월일이 다음날로 넘어가는 것을 막기 위해서 3시간씩 앞으로 땡겨줍니다. 마지막에 다시 더해줘야 합니다.
    delta = datetime.timedelta(hours=3)
    train['방송일시'] = train.방송일시.apply(lambda x: x - delta)

    # dayofweek는 날짜에서 요일(월~일)을 가져오는 기능입니다.
    # 값은 0(월), 1(화), 2(수), 3(목), 4(금), 5(토), 6(일) 을 나타냅니다.
    train["방송일시_dow"] = train["방송일시"].dt.dayofweek

    # 요일 외에 다른 정보를 가져오기 위해서 다시 string으로 바꿔줍니다.
    train['방송일시'] = train.방송일시.astype(str)

    # MMDDhhmm 정보를 가져옵니다.
    train['방송일시_MM'] = train['방송일시'].apply(lambda x: x[5:7])
    train['방송일시_DD'] = train['방송일시'].apply(lambda x: x[8:10])
    train['방송일시_hh'] = train['방송일시'].apply(lambda x: x[11:13])
    train['방송일시_mm'] = train['방송일시'].apply(lambda x: x[14:16])

    # schedule 관련 칼럼을 더 만들어보겠습니다. # MMDD # DDHH # HHMM # weekday/weekends
    train['방송일시_MMDD'] = train['방송일시_MM'] + train['방송일시_DD']
    train['방송일시_DDhh'] = train['방송일시_DD'] + train['방송일시_hh']
    train['방송일시_hhmm'] = train['방송일시_hh'] + train['방송일시_mm']
    train['방송일시_MMDDhh'] = train['방송일시_MM'] + train['방송일시_DD'] + train['방송일시_hh']

    # mmmm_1 은 1일 사이클로 분단위로 환산한 것
    # mmmm_2 는 1달 사이클로 분단위로 환산한 것
    # mmmm_3 는 1년 사이클로 분단위로 환산한 것
    train['방송일시_mmmm_1'] = train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60
    train['방송일시_mmmm_2'] = train['방송일시_DD'].astype(int) * train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60
    train['방송일시_mmmm_3'] = train['방송일시_MM'].astype(int) * train['방송일시_DD'].astype(int) * train['방송일시_hh'].astype(int) * train['방송일시_mm'].astype(int) * 60

    # weekday = 1 / weekends = 0
    train.loc[train.방송일시_dow == 5, '방송일시_dow2'] = '0'
    train.loc[train.방송일시_dow == 6, '방송일시_dow2'] = '0'
    train['방송일시_dow2'] = train.방송일시_dow2.fillna('1')
    
    #다이어트
    train['방송일시_dow'] = train.방송일시_dow.astype(np.int16)
    train['방송일시_MM'] = train.방송일시_MM.astype('category')
    train['방송일시_DD'] = train.방송일시_DD.astype('category')
    train['방송일시_hh'] = train.방송일시_hh.astype('category')
    train['방송일시_mm'] = train.방송일시_mm.astype('category')
    train['방송일시_MMDD'] = train.방송일시_MMDD.astype('category')
    train['방송일시_DDhh'] = train.방송일시_DDhh.astype('category')
    train['방송일시_hhmm'] = train.방송일시_hhmm.astype('category')
    train['방송일시_dow2'] = train.방송일시_dow2.astype('category')
    
    return train

In [ ]:
import datetime
df_all = preprocessing_4(df_all)

In [ ]:
def pre_nlp(train):
    # 제품명에서 불필요한 요소 전처리
    new_product_name = []

    re_stop = re.compile("""\([가-힣]{1,5}\+[가-힣]{1,5}\)|\(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?|무료체험|
    |포함|국내[가-힣]+\s|무료설치|\s?신제품\s?|\s?패키지\s?|[0-9]+종|풀코디|set|SET|풀세트|[0-9]+세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}|[0-9]{1,2}인용|[0-9]{1,2}박스|[0-9\.]{1,4}미터|[0-9\.]{1,4}kg|[0-9\.]{1,4}[kKgG]|[0-9]{1,3}[벌롤종단구대P개통병포미봉팩장gL매]|
    |\s[0-9\.]{1,4}[Mm]|\(.{1,10}\)$|^\(.?\)|^[0-9]{2,4}\s|\s+[0-9]{2,4}\s|[0-9]{2,4}년\s|[0-9]{2,4}년형\s|시즌[0-9]|[0-9]{1,3}\%|\(.{1,20}\)$|기본형|고급형|오리지널|[대중소大中小]형|.{1,10}by|,|\s?총[0-9\s]|^[가-힣a-zA-Z]{3,3}의|
    |S\/S|F\/W|f\/w|s\/s|[가-힣]+형\s|[슈퍼]{0,2}싱글|\s[SQK퀸킹]{1,2}\s|[SQK퀸킹]{1,2}$|[가-힣]{0,2}사이즈""")

    remnants = re.compile('[",g\+lL-]\s|ml|_|\sx|[\[\]\］!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]|\s종$|\s[0-9]+\s|\s{2,}')

    for i in train.상품명:
        tmp = re_stop.sub(' ',i)
        tmp = tmp.strip()
        tmp = remnants.sub(' ',tmp)
        new_product_name.append(tmp.strip())
    
    train['new_상품명'] = new_product_name
    
    # 상품명에서 브랜드 추출
    train['상품명_brand'] = train['new_상품명'].apply(lambda x: x[:2])
    
    return train

In [ ]:
import re
df_all = pre_nlp(df_all)

In [ ]:
naver = pd.read_csv('/gdrive/My Drive/naver.csv')

In [ ]:
def outside_feature_naver(train, naver):
    # 먼저 합쳐줍니다.
    train = pd.merge(train, naver, on='상품명', how='left')
    
    # 전처리 해줍니다.
    train['review_counts'] = train['review_counts'].fillna(0)
    train['internet_price'] = train['internet_price'].fillna(0)
    train['review_counts'] = train['review_counts'].astype(str)
    train['internet_price'] = train['internet_price'].astype(str)
    train['review_counts'] = train['review_counts'].apply(lambda x: x.replace(',',''))
    train['internet_price'] = train['internet_price'].apply(lambda x: x.replace(',',''))
    train['internet_price'] = train['internet_price'].apply(lambda x: x.replace('원',''))
    train['review_counts'] = train['review_counts'].astype(int)
    train['internet_price'] = train['internet_price'].astype(int)

    # 오차를 컬럼으로 만들어줍니다.
    train['price_minus'] = train['판매단가'] - train['internet_price']

    # 네이버에 서칭이 되는지 여부를 알려줍니다.
    train['search_naver'] = train['internet_price'] + train['review_counts']
    train['search_naver'] = train['search_naver'].apply(lambda x: 0 if x == 0 else 1)
    
    return train

In [ ]:
df_all = outside_feature_naver(df_all, naver)

In [ ]:
# 기상청 통계 파일 읽기
weather = pd.read_csv('/gdrive/My Drive/OBS_ASOS_TIM_20200902192155.csv', encoding='latin1')
columns = ['location_id','location','date','temperature']
weather.columns = columns

In [ ]:
def outside_feature_weather(weather):
    # datetime으로 바꿔줍니다.
    weather['date'] = pd.to_datetime(weather['date'], errors='coerce')

    # 월일이 다음날로 넘어가는 것을 막기 위해서 3시간씩 앞으로 땡겨줍니다. 마지막에 다시 더해줘야 합니다.
    delta = datetime.timedelta(hours=3)
    weather['date'] = weather.date.apply(lambda x: x - delta)

    # 요일 외에 다른 정보를 가져오기 위해서 다시 string으로 바꿔줍니다.
    weather['date'] = weather.date.astype(str)
    
    # key column 만들어주기.
    weather['방송일시_MMDDhh'] = weather['date'].apply(lambda x: x[5:7]) + weather['date'].apply(lambda x: x[8:10]) + weather['date'].apply(lambda x: x[11:13])

    # 필요없는 칼럼은 미리 버려줍니다.
    weather.drop(['location_id', 'location', 'date'], axis=1, inplace=True)
    
    return weather

In [ ]:
weather = outside_feature_weather(weather)

# 합쳐줍니다.
df_all = pd.merge(df_all, weather, on='방송일시_MMDDhh', how='left')

In [ ]:
ratings = pd.read_csv(r'/gdrive/My Drive/tv_new.csv')
# 합치기 좋게 행렬 축을 바꿔줍니다.
ratings_melt = pd.melt(ratings, id_vars=['시간대'])
# 평균 시청률이 있습니다. 없애줍니다.
ratings_melt.drop(ratings_melt[ratings_melt['variable'].str.contains('to')].index, axis=0, inplace=True)
ratings_melt.drop(ratings_melt[ratings_melt['시간대'].str.contains('월')].index, axis=0, inplace=True)

In [ ]:
change = 0
change_label = []

for i in range(len(df_all)):
    if i == 0: # 첫번째 행은 이걸로 하십쇼.
        change = 1 # 방송 시작이니 1을 줍니다.
        change_label.append(change) # 그리고 어펜드.

    elif df_all['마더코드'][i] == df_all['마더코드'][i-1]: # 자기의 마더코드가 앞행의 것과 같을 때
        change = 0 # 기존 방송이니 0을 줍니다.
        change_label.append(change) # 그리고 어펜드.
        
    else: # 근데 자기 마더코드와 앞행의 마더코드가 다르다면, 새로운 방송이 시작된 거겠죠?
        change = 1 # 방송 시작이니 1을 줍니다.
        change_label.append(change) # 그리고 어펜드.

df_all['change_label'] = change_label

In [ ]:
# 시청률 df는 편하게 change_label 을 그냥 1로 때려박습니다.
ratings_melt['change_label'] = 1

In [ ]:
# train의 방송일시를 datetime 으로 바꿔서 키 값으로 줍니다.
df_all['ratings_key'] = pd.to_datetime(df_all.방송일시)
# ratings 도 마찬가지.
ratings_melt['ratings_key'] = pd.to_datetime((ratings_melt.variable + " " + ratings_melt.시간대))
# train이 3시간 당겨져 있으니, ratings도 3시간 당겨줍니다.
delta = datetime.timedelta(hours=3)
ratings_melt['ratings_key'] = ratings_melt.ratings_key.apply(lambda x: x - delta)

In [ ]:
# 합쳐줍니다.
df_all = pd.merge(df_all, ratings_melt[['ratings_key','change_label', 'value']], on=['change_label','ratings_key'], how='left')

In [ ]:
# 방송 시작 시청률만 넣어주기 때문에 nan값이 생깁니다. 전부 ffill로 채워주면 점수가 떨어지고 0을 넣어주면 올라갑니다... 
df_all.value = df_all.value.fillna(0)

In [ ]:
# 키값은 드랍
df_all.drop('ratings_key', axis=1, inplace=True)
# 칼럼명은 변경
df_all.rename(columns = {'value':'ratings_start'}, inplace=True)

In [ ]:
# 가격이 끝에 9xxxx 으로 끝나는 애들을 따로 피쳐로 빼봅시다.

df_all['판매단가'] = df_all['판매단가'].astype(str)
df_all.loc[df_all['판매단가'].str.contains('900') == True, "가격_9x"] = '1'
df_all.loc[df_all['판매단가'].str.contains('900') == False, "가격_9x"] = '0'
df_all['판매단가'] = df_all['판매단가'].astype(int)

In [ ]:
def stock_crawler(code):

    for c, i in zip(code, range(len(code))):
        
        url = 'https://finance.naver.com/item/sise_day.nhn' # URL은 고정
        result = []
        tdtext_list = ['날짜', c]

        for num in range(16, 43): # 페이지 범위 지정
            params = {'code' : c, # 지수 코드 지정
                      'page' : f'{num}'}

            resp = requests.get(url, params = params)
            soup = BeautifulSoup(resp.content, 'html.parser')

            for tr in soup.find_all('tr')[2:14]:
                new_dict = { }
                td_list = tr.find_all('td')
                if not td_list[0].text.strip():
                    continue
                for i in range(2):
                    new_dict[tdtext_list[i]] = td_list[i].text.strip()
                result.append(new_dict)

        globals()['stock_{}'.format(c)] = pd.DataFrame(result, columns = tdtext_list)

In [ ]:
import requests
from bs4 import BeautifulSoup
code = ['143860', '098560', '228790', '228800', '227560', '227550', '307510']
stock_crawler(code)

In [ ]:
# 다 합쳐줍니다. 함수로 하고 싶은데,너무 귀찮습니다. 알고리즘 짜기. 그냥 노가다 뜁니다.
stock_all = pd.merge(stock_098560, stock_143860, on='날짜')
stock_all = pd.merge(stock_all, stock_227550, on='날짜')
stock_all = pd.merge(stock_all, stock_227560, on='날짜')
stock_all = pd.merge(stock_all, stock_228790, on='날짜')
stock_all = pd.merge(stock_all, stock_228800, on='날짜')
stock_all = pd.merge(stock_all, stock_307510, on='날짜')

# key값을 만들기 위해 귀찮지만 형변환 두 단계
stock_all['날짜'] = pd.to_datetime(stock_all['날짜'], errors='coerce')
stock_all['날짜'] = stock_all['날짜'].astype(str)

# 1월 1일 자료가 없는 관계로 그 이전 종가를 가져온다.
stock_all.replace({'2018-12-28': '2019-01-01'}, inplace=True)

# 컴퓨터가 인식하도록 int 로 수치를 바꿔줍니다.

for col in stock_all.columns:
    stock_all[col] = stock_all[col].apply(lambda x: x.replace(',',''))
    if col == '날짜':
        pass
    else:
        stock_all[col] = stock_all[col].astype(np.int16)

In [ ]:
# 귀찮지만 train에도 만들어주기.
df_all['날짜'] = df_all['방송일시'].apply(lambda x: x[:10])
df_all['날짜'] = df_all['날짜'].astype(str)

df_all = pd.merge(df_all, stock_all, on='날짜', how='left')

In [ ]:
df_all= df_all.fillna(method='ffill')
df_all.drop(['날짜'], axis=1, inplace=True)

In [ ]:
# 상품별 최대 판매단가 추가

tmp = df_all[['상품명','판매단가']].groupby('상품명').max()[['판매단가']].reset_index()
df_all = pd.merge(df_all, tmp, on='상품명', how='left')
df_all.rename({'판매단가_x':'판매단가','판매단가_y':'할인'}, axis=1, inplace= True)
df_all['할인여부'] = df_all['할인'] - df_all['판매단가']

df_all.drop(['할인'], axis=1, inplace=True)

In [ ]:
# 시간대별 평균 취급액 추가(train default 변수)

tmp = df_all.groupby('방송일시_hhmm').mean()[['취급액']].reset_index()
df_all = pd.merge(df_all, tmp, on='방송일시_hhmm', how='left')
df_all.rename({'취급액_y':'mean_amt_by_hhmm','취급액_x':'취급액'}, axis=1, inplace=True)

In [ ]:
# 시간대별로 4파트로 나눕니다. 아래는 민수의 구분

list_0 = ['03', '04', '05', '06', '07','08'] # 낮12시 이전
list_1 = ['09', '10', '11', '12', '13','14', '15', '16', '17'] # 낮 12시부터
list_2 = ['18', '19', '20', '21', '22', '23'] # 밤 9시부터

# 시간대별로 4파트로 나눕니다. 아래는 왕수의 구분

list_4 = ['03', '21', '22', '23'] # 심야시간대, 03, 21, 22, 23
list_5 = ['04', '05', '06', '07', '08', '09'] # 낮 12시까지 오전
list_6 = ['10', '11', '12', '13', '14', '15'] # 저녁 18시 이전까지 오후, 나머진 저녁

In [ ]:
df_all['time_cat'] = df_all['방송일시_hh'].apply(lambda x:0 if x in list_0 else(1 if x in list_1 else (2 if x in list_2 else 3)))
df_all['time_cat2'] = df_all['방송일시_hh'].apply(lambda x:0 if x in list_4 else(1 if x in list_5 else (2 if x in list_6 else 3)))

In [ ]:
# 판매단가 카테고리화

df_all['판매단가_cat'] = df_all['판매단가'].apply(lambda x: 0 if x <= 15000 else (1 if x<=30000 else (2 if x <=45000 else (3 if x <= 60000 else (4 if x <= 100000 else 5)))))
df_all['판매단가_cat'].value_counts()

5    20731
4     6964
3     6588
2     5054
1      748
0        3
Name: 판매단가_cat, dtype: int64

In [ ]:
kospi = pd.read_csv('/gdrive/My Drive/코스피지수 내역.csv')
kospi.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 30일,"2,197.67","2,202.62","2,208.04","2,195.65",416.40M,-0.30%
1,2019년 12월 27일,"2,204.21","2,183.70","2,215.55","2,177.40",490.17M,0.29%
2,2019년 12월 26일,"2,197.93","2,192.22","2,198.01","2,183.90",571.31M,0.36%
3,2019년 12월 24일,"2,190.08","2,206.23","2,206.23","2,187.58",569.97M,-0.62%
4,2019년 12월 23일,"2,203.71","2,208.22","2,209.20","2,196.43",502.03M,-0.02%


In [ ]:
kospi = kospi[['날짜','변동 %']]
kospi['변동 %'] = kospi['변동 %'].apply(lambda x: x[:-1])
kospi['변동 %'] = kospi['변동 %'].astype(float)

kospi['날짜'] = kospi['날짜'].apply(lambda x: x[6:12])
kospi['날짜'] = kospi['날짜'].apply(lambda x: x.replace('월 ',''))

In [ ]:
df_all = pd.merge(df_all, kospi, left_on = '방송일시_MMDD',right_on = '날짜',how='left')
del df_all['날짜']
df_all['변동 %'] = df_all['변동 %'].fillna(0)

In [ ]:
# 같은 시간에 방송한거 same 이름으로 묶어줌
df_all['same'] = 1
for i in range(1,len(df_all)):
  if df_all['cast_count'][i] == df_all['cast_count'][i-1]:
    df_all['same'][i] = df_all['same'][i-1]
  else:
    df_all['same'][i] = df_all['same'][i-1] + 1

In [ ]:
# 같은 시간에 여러 품목 방송했는지 알기 위해 cast_count_com 변수 새로 생성
df_all = pd.merge(df_all, df_all.groupby('same').sum().reset_index()[['same','cast_count']], on ='same', how='left')
df_all.rename({'cast_count_x':'cast_count', 'cast_count_y':'cast_count_com'}, axis=1, inplace= True)

# 최종 비교 위해 com 또 생성
df_all['com'] = df_all['cast_count'] - df_all['cast_count_com']

# 같은 방송 여러 품목에 fw라는 순번을 붙여줌
tmp = df_all.copy()
tmp = tmp[tmp['com'] != 0]
tmp.reset_index(drop=True, inplace=True)
tmp['fw'] = 1
for i in range(1, len(tmp)):
  if tmp['same'][i] == tmp['same'][i-1]:
    tmp['fw'][i] = tmp['fw'][i-1]+1
  else:
    pass

In [ ]:
# 다시 train 데이터에 fw 붙여주기 fake_weight3으로 이름 붙이기
df_all = pd.merge(df_all, tmp[['방송일시','상품명','fw']], on =['방송일시','상품명'], how = 'left')
df_all.rename({'fw':'fake_weight3'},axis=1,inplace=True)
df_all.drop(['same','cast_count_com'], axis=1, inplace=True)
#train['fw'] = train['fw'].astype(int)

In [ ]:
df_all['fake_weight3']= df_all['fake_weight3'].fillna(0)
df_all['fake_weight3'] = df_all['fake_weight3'].astype(int)

In [ ]:
# 한 방송에 여러상품 포함하면 1 아니면 0
df_all['com'] = df_all['com'].apply(lambda x: 1 if x==0 else 0)
df_all['com'].value_counts()

0    26335
1    13753
Name: com, dtype: int64

In [ ]:
search = pd.read_excel('/gdrive/My Drive/datalab.xlsx',skiprows=6)
search.head()

,날짜,NS홈쇼핑 NS,날짜.1,롯데홈쇼핑,날짜.2,GS홈쇼핑,날짜.3,현대홈쇼핑,날짜.4,CJ홈쇼핑
0,2019-01-01,17.30971,2019-01-01,56.97459,2019-01-01,38.37021,2019-01-01,50.46388,2019-01-01,8.50646
1,2019-01-02,17.66025,2019-01-02,60.25098,2019-01-02,37.89114,2019-01-02,46.27374,2019-01-02,7.57869
2,2019-01-03,14.87696,2019-01-03,62.30282,2019-01-03,37.54060,2019-01-03,49.87263,2019-01-03,7.12766
3,2019-01-04,13.96087,2019-01-04,59.56159,2019-01-04,37.43310,2019-01-04,49.71606,2019-01-04,6.97576
4,2019-01-05,14.40957,2019-01-05,65.72410,2019-01-05,50.29562,2019-01-05,63.80313,2019-01-05,8.42466


In [ ]:
search = search[['날짜','NS홈쇼핑 NS']]
search['날짜'] = search['날짜'].apply(lambda x: x[5:])
search['날짜'] = search['날짜'].apply(lambda x: x.replace('-',''))
search.head()

,날짜,NS홈쇼핑 NS
0,0101,17.30971
1,0102,17.66025
2,0103,14.87696
3,0104,13.96087
4,0105,14.40957


In [ ]:
df_all = pd.merge(df_all, search, left_on ='방송일시_MMDD', right_on = '날짜', how = 'left')
df_all.rename({'NS홈쇼핑 NS':'search_compare'},axis=1, inplace= True)

In [ ]:
# 라벨인코딩 숫자가 많아지니 함수로 만들어버립니다.
def label_encoding(train_, columns, number):
    length = range(number)
    for c, i in zip(columns, length):
        # 라벨인코더 피팅
        globals()['encoder_{}:'.format(i)] = LabelEncoder()
        globals()['encoder_{}:'.format(i)].fit(train_[c].values)
        
        # 트레인셋에 트랜스폼
        train_['encoding_{}:'.format(c)] = globals()['encoder_{}:'.format(i)].transform(train_[c])

In [ ]:
# 인코딩할 컬럼을 리스트로 넣어줍니다.
columns = ['상품명', '상품군', '상품명_brand', 'new_상품명','cat1','cat2','cat3']
number = len(columns)

# 함수로 바로 처리.
label_encoding(df_all, columns, number)

In [ ]:
df_all.drop(['index'], axis=1, inplace=True)

In [ ]:

df_all['상품명_kid'] = df_all['상품명_kid'].astype(int)
df_all['가격_9x'] = df_all['가격_9x'].astype(int)

train = df_all[:37372]
test = df_all[37372:]

In [ ]:
# 예측 값 추정이 불가능한 취급액 0원인 애들 제거해줍니다.

train = train[train.취급액 != 0]
train.shape

(35379, 72)

In [ ]:
# train 셋의 아웃라이어를 제거해봅니다...1억 이상의 취급액을 가진 데이터가 약 1% 정도 있습니다.

train = train[train.취급액 < 100000000]
train.shape

(35196, 72)

In [ ]:
train_, test_ = train_test_split(train, test_size=0.2, random_state=529)

In [ ]:
features = ['노출(분)', '마더코드', '상품코드','판매단가',
       'cast_time', 'cast_count', 'cast_time_sum', 'cast_count_sum',
       'cast_time_ratio', '상품명_plan', '상품명_add', '상품명_maker', '상품명_set',
       '상품명_sex', '상품명_kid', 'fake_weight', 'fake_weight2', 'ratings_mean',
       '방송일시_dow', '방송일시_MM', '방송일시_DD', '방송일시_hh',
       '방송일시_mm', '방송일시_MMDD', '방송일시_DDhh', '방송일시_hhmm', '방송일시_MMDDhh',
       '방송일시_mmmm_1', '방송일시_mmmm_2', '방송일시_mmmm_3', '방송일시_dow2', 'review_counts', 'internet_price', 'price_minus',
       'search_naver', 'temperature', 'change_label', 'ratings_start', '가격_9x',
       '할인여부', 'mean_amt_by_hhmm', 'time_cat', 'time_cat2', '판매단가_cat',
       'encoding_상품명:', 'encoding_상품군:', 'encoding_상품명_brand:',
       'encoding_new_상품명:','encoding_cat1:', 'encoding_cat2:','encoding_cat3:',
       '변동 %', 'com', 'fake_weight3','search_compare'
           ]

label = '취급액'

In [ ]:
len(features)

55

In [ ]:
# rating_mean 은 일단 제외하고 학습합니다.
# 변수 순서가 바뀌면 점수가 바뀝니다.

X_train = train_[features].values 
y_train = train_[label].values
X_test = test_[features].values 
y_test = test_[label].values

In [ ]:
y_train_log = np.log(y_train)

In [ ]:
rf = RandomForestRegressor(n_jobs=3,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

In [ ]:
def mape(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

36.56769016000426

In [ ]:
!pip install boruta

     |████████████████████████████████| 61kB 1.8MB/s 


In [ ]:
from boruta import BorutaPy

cols = train.columns
x = train[features].values
y = train[label].values
log_y = np.log(y)
#  randomforest 및 boruta 객체 형성
rf = RandomForestRegressor()
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=529)
feat_selector.fit(x, log_y)

# 변수선택 이후 데이터 형성
X_filtered = feat_selector.transform(x)

x_train, x_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.2, random_state=529)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

# # Boruta 적용 결과 저장
# new_cols = []
# for x in range(len(feat_selector.support_)):
#     if feat_selector.support_[x] == True:
#         new_cols.append(cols[x])

# # 결과 확인
# print('이용 컬럼 :{}'.format(new_cols))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	55
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	24
Tentative: 	3
Rejected: 	28
Iteration: 	9 / 100
Confirmed: 	24
Tentative: 	3
Rejected: 	28
Iteration: 	10 / 100
Confirmed: 	24
Tentative: 	3
Rejected: 	28
Iteration: 	11 / 100
Confirmed: 	24
Tentative: 	3
Rejected: 	28
Iteration: 	12 / 100
Confirmed: 	24
Tentative: 	3
Rejected: 	28
Iteration: 	13 / 100
Confirmed: 	24
Tentative: 	2
Rejected: 	29
Iteration: 	14 / 100
Confirmed: 	24
Tentative: 	2
Rejected: 	29
Iteration: 	15 / 100
Confirmed: 	24
Tentative: 	2
Rejected: 	29
Iteration: 	16 / 100
Confirmed: 	24
Tentative: 	2
Reject

In [ ]:
# Boruta 적용 결과 저장
new_cols = []
cols = train[features].columns
for x in range(len(feat_selector.support_)):
    if feat_selector.support_[x] == True:
        new_cols.append(cols[x])

# 결과 확인
print('이용 컬럼 :{}'.format(new_cols))

이용 컬럼 :['노출(분)', '마더코드', '상품코드', '판매단가', 'cast_time', 'cast_time_ratio', '상품명_maker', '방송일시_dow', '방송일시_MMDD', '방송일시_hhmm', '방송일시_MMDDhh', 'review_counts', 'internet_price', 'price_minus', 'temperature', 'mean_amt_by_hhmm', 'time_cat2', 'encoding_상품명:', 'encoding_상품군:', 'encoding_new_상품명:', 'encoding_cat1:', 'encoding_cat2:', 'encoding_cat3:', 'com', 'fake_weight3', 'search_compare']


In [ ]:
# rating_mean 은 일단 제외하고 학습합니다.
# 변수 순서가 바뀌면 점수가 바뀝니다.

X_train = train_[new_cols].values 
y_train = train_[label].values
X_test = test_[new_cols].values 
y_test = test_[label].values

In [ ]:
y_train_log = np.log(y_train)

In [ ]:
rf = RandomForestRegressor(n_jobs=3,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

In [ ]:
pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

36.90556334695656

In [ ]:
len(new_cols)

26

In [ ]:
new_cols_boruta = new_cols.copy()

In [ ]:
from sklearn.feature_selection import SelectFromModel
# rating_mean 은 일단 제외하고 학습합니다.
# 변수 순서가 바뀌면 점수가 바뀝니다.

X_train = train_[features].values 
y_train = train_[label].values
X_test = test_[features].values 
y_test = test_[label].values

y_train_log = np.log(y_train)

x = train[features].values
y = train[label].values
y_log = np.log(y)

In [ ]:
sel = SelectFromModel(RandomForestRegressor(random_state=529))
sel.fit(x, y_log)

SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                                criterion='mse', max_depth=None,
                                                max_features='auto',
                                                max_leaf_nodes=None,
                                                max_samples=None,
                                                min_impurity_decrease=0.0,
                                                min_impurity_split=None,
                                                min_samples_leaf=1,
                                                min_samples_split=2,
                                                min_weight_fraction_leaf=0.0,
                                                n_estimators=100, n_jobs=None,
                                                oob_score=False,
                                                random_state=529, verbose=0,
                                                warm

In [ ]:
sel.get_support()

array([False, False, False,  True,  True, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False,  True, False,  True,
       False, False, False, False, False,  True, False,  True,  True,
       False])

In [ ]:
selected_feat= train[features].columns[(sel.get_support())]
len(selected_feat)

10

In [ ]:
selected_feat.values.tolist()

['판매단가',
 'cast_time',
 'cast_time_ratio',
 'internet_price',
 'mean_amt_by_hhmm',
 'time_cat2',
 'encoding_상품명:',
 'encoding_cat3:',
 'com',
 'fake_weight3']

In [ ]:
new_cols_sklearn = selected_feat.values.tolist()

In [ ]:
# rating_mean 은 일단 제외하고 학습합니다.
# 변수 순서가 바뀌면 점수가 바뀝니다.

X_train = train_[new_cols_sklearn].values 
y_train = train_[label].values
X_test = test_[new_cols_sklearn].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

In [ ]:
rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

In [ ]:
pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

40.18191883550131

In [ ]:
# recursive feature elimination

from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(), 10)

In [ ]:
fit = rfe.fit(x, y_log)
print( fit.n_features_) 
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

10
Selected Features: [False False False  True  True False False False  True False False False
 False False False False False False False False False False False False
 False False  True False False False False False  True False False False
 False False False False  True False False False  True False False False
 False False  True False  True  True False]
Feature Ranking: [17 18  3  1  1 33 40 30  1 26 28 22 38 32 46 29 45 19 13 35 23 27 41 15
 11  7  1 25 21 14 31 12  1 10 44  6 37 43 36 34  1 42  4 39  1 16 24  5
 20  8  1  9  1  1  2]


In [ ]:
ord = [3,4,8,26,32,40,44,50,52,53]
new_cols_RFE = []
for i in ord:
  new_cols_RFE.append(features[i])
new_cols_RFE

['판매단가',
 'cast_time',
 'cast_time_ratio',
 '방송일시_MMDDhh',
 'internet_price',
 'mean_amt_by_hhmm',
 'encoding_상품명:',
 'encoding_cat3:',
 'com',
 'fake_weight3']

In [ ]:
X_train = train_[new_cols_RFE].values 
y_train = train_[label].values
X_test = test_[new_cols_RFE].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

In [ ]:
rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

39.34033606472361

In [ ]:
x = train[features].values
y = train[label].values
y_log = np.log(y)

In [ ]:
# recursive feature elimination

from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(), 15)

In [ ]:
fit = rfe.fit(x, y_log)
print( fit.n_features_) 
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

15
Selected Features: [False False  True  True  True False False False  True False False False
 False False False False False False False False False False False False
 False False  True False False False False False  True False False  True
 False False False False  True False  True False  True False False  True
 False False  True False  True  True  True]
Feature Ranking: [12 13  1  1  1 28 35 25  1 21 23 17 33 29 40 24 41 14  8 31 18 22 36 10
  6  2  1 20 16  9 26  7  1  5 39  1 27 38 32 30  1 37  1 34  1 11 19  1
 15  3  1  4  1  1  1]


In [ ]:
ord = [2,3,4,8,26,32,35,40,42,44,47,50,52,53,54]
new_cols_RFE = []
for i in ord:
  new_cols_RFE.append(features[i])
new_cols_RFE

['상품코드',
 '판매단가',
 'cast_time',
 'cast_time_ratio',
 '방송일시_MMDDhh',
 'internet_price',
 'temperature',
 'mean_amt_by_hhmm',
 'time_cat2',
 'encoding_상품명:',
 'encoding_new_상품명:',
 'encoding_cat3:',
 'com',
 'fake_weight3',
 'search_compare']

In [ ]:
X_train = train_[new_cols_RFE].values 
y_train = train_[label].values
X_test = test_[new_cols_RFE].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

38.777473540006525

In [ ]:
# recursive feature elimination

from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(), 20)

In [ ]:
fit = rfe.fit(x, y_log)
print( fit.n_features_) 
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

20
Selected Features: [False False  True  True  True False False False  True False False False
 False False False False False False False False False False False False
  True  True  True False False False False False  True  True False  True
 False False False False  True False  True False  True False False  True
 False  True  True  True  True  True  True]
Feature Ranking: [ 7  8  1  1  1 22 30 20  1 15 18 12 28 23 35 19 36 10  3 26 13 17 31  5
  1  1  1 16 11  4 21  2  1  1 34  1 25 33 27 24  1 32  1 29  1  6 14  1
  9  1  1  1  1  1  1]


In [ ]:
ord = [2,3,4,8,24,25,26,32,33,35,40,42,44,47,49,50,51,52,53,54]
print(len(ord))
new_cols_RFE = []
for i in ord:
  new_cols_RFE.append(features[i])
print(new_cols_RFE)

20
['상품코드', '판매단가', 'cast_time', 'cast_time_ratio', '방송일시_DDhh', '방송일시_hhmm', '방송일시_MMDDhh', 'internet_price', 'price_minus', 'temperature', 'mean_amt_by_hhmm', 'time_cat2', 'encoding_상품명:', 'encoding_new_상품명:', 'encoding_cat2:', 'encoding_cat3:', '변동 %', 'com', 'fake_weight3', 'search_compare']


In [ ]:
X_train = train_[new_cols_RFE].values 
y_train = train_[label].values
X_test = test_[new_cols_RFE].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

37.79954608000143

In [ ]:
# recursive feature elimination

from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(), 25)

In [ ]:
fit = rfe.fit(x, y_log)
print( fit.n_features_) 
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

25
Selected Features: [False False  True  True  True False False False  True False False False
 False False False False False False  True False False False False  True
  True  True  True False False  True False  True  True  True False  True
 False False False False  True False  True False  True  True False  True
 False  True  True  True  True  True  True]
Feature Ranking: [ 2  3  1  1  1 17 25 15  1  9 13  7 23 18 30 14 31  4  1 22  8 12 26  1
  1  1  1 11  6  1 16  1  1  1 29  1 19 28 21 20  1 27  1 24  1  1 10  1
  5  1  1  1  1  1  1]


In [ ]:
ord = [2,3,4,8,18,23,24,25,26,29,31,32,33,35,40,42,44,45,47,49,50,51,52,53,54]
print(len(ord))
new_cols_RFE = []
for i in ord:
  new_cols_RFE.append(features[i])
print(new_cols_RFE)

25
['상품코드', '판매단가', 'cast_time', 'cast_time_ratio', '방송일시_dow', '방송일시_MMDD', '방송일시_DDhh', '방송일시_hhmm', '방송일시_MMDDhh', '방송일시_mmmm_3', 'review_counts', 'internet_price', 'price_minus', 'temperature', 'mean_amt_by_hhmm', 'time_cat2', 'encoding_상품명:', 'encoding_상품군:', 'encoding_new_상품명:', 'encoding_cat2:', 'encoding_cat3:', '변동 %', 'com', 'fake_weight3', 'search_compare']


In [ ]:
X_train = train_[new_cols_RFE].values 
y_train = train_[label].values
X_test = test_[new_cols_RFE].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

37.360352541540166

In [ ]:
# recursive feature elimination

from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(), 30)

fit = rfe.fit(x, y_log)
print( fit.n_features_) 
print("Selected Features: %s"% fit.support_) 
print("Feature Ranking: %s"% fit.ranking_)

30
Selected Features: [ True  True  True  True  True False False False  True False False False
 False False False False False  True  True False False False False  True
  True  True  True False  True  True False  True  True  True False  True
 False False False False  True False  True False  True  True False  True
  True  True  True  True  True  True  True]
Feature Ranking: [ 1  1  1  1  1 12 20 10  1  5  7  2 17 13 26  9 25  1  1 15  3  8 21  1
  1  1  1  6  1  1 11  1  1  1 24  1 14 23 18 16  1 22  1 19  1  1  4  1
  1  1  1  1  1  1  1]


In [ ]:
ord = [0,1,2,3,4,8,17,18,23,24,25,26,28,29,31,32,33,35,40,42,44,45,47,48,49,50,51,52,53,54]
print(len(ord))
new_cols_RFE = []
for i in ord:
  new_cols_RFE.append(features[i])
print(new_cols_RFE)

30
['노출(분)', '마더코드', '상품코드', '판매단가', 'cast_time', 'cast_time_ratio', 'ratings_mean', '방송일시_dow', '방송일시_MMDD', '방송일시_DDhh', '방송일시_hhmm', '방송일시_MMDDhh', '방송일시_mmmm_2', '방송일시_mmmm_3', 'review_counts', 'internet_price', 'price_minus', 'temperature', 'mean_amt_by_hhmm', 'time_cat2', 'encoding_상품명:', 'encoding_상품군:', 'encoding_new_상품명:', 'encoding_cat1:', 'encoding_cat2:', 'encoding_cat3:', '변동 %', 'com', 'fake_weight3', 'search_compare']


In [ ]:
X_train = train_[new_cols_RFE].values 
y_train = train_[label].values
X_test = test_[new_cols_RFE].values 
y_test = test_[label].values
y_train_log = np.log(y_train)

rf = RandomForestRegressor(n_jobs=-1,
                           random_state=592)

rf.fit(X_train, y_train_log)
pred_log = rf.predict(X_test)

pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

37.10062044236769

In [ ]:
# class GeneticSelector():
#     def __init__(self, estimator, n_gen, size, n_best, n_rand, 
#                  n_children, mutation_rate):
#         # Estimator 
#         self.estimator = estimator
#         # Number of generations
#         self.n_gen = n_gen
#         # Number of chromosomes in population
#         self.size = size
#         # Number of best chromosomes to select
#         self.n_best = n_best
#         # Number of random chromosomes to select
#         self.n_rand = n_rand
#         # Number of children created during crossover
#         self.n_children = n_children
#         # Probablity of chromosome mutation
#         self.mutation_rate = mutation_rate
        
#         if int((self.n_best + self.n_rand) / 2) * self.n_children != self.size:
#             raise ValueError("The population size is not stable.")  
            
#     def initilize(self):
#         population = []
#         for i in range(self.size):
#             chromosome = np.ones(self.n_features, dtype=np.bool)
#             mask = np.random.rand(len(chromosome)) < 0.3
#             chromosome[mask] = False
#             population.append(chromosome)
#         return population

#     def fitness(self, population):
#         X, y = self.dataset
#         scores = []
#         for chromosome in population:
#             score = mape(X[:,chromosome], y)
#             scores.append(score)
#         scores, population = np.array(scores), np.array(population) 
#         inds = np.argsort(scores)
#         return list(scores[inds]), list(population[inds,:])

#     def select(self, population_sorted):
#         population_next = []
#         for i in range(self.n_best):
#             population_next.append(population_sorted[i])
#         for i in range(self.n_rand):
#             population_next.append(random.choice(population_sorted))
#         random.shuffle(population_next)
#         return population_next

#     def crossover(self, population):
#         population_next = []
#         for i in range(int(len(population)/2)):
#             for j in range(self.n_children):
#                 chromosome1, chromosome2 = population[i], population[len(population)-1-i]
#                 child = chromosome1
#                 mask = np.random.rand(len(child)) > 0.5
#                 child[mask] = chromosome2[mask]
#                 population_next.append(child)
#         return population_next
	
#     def mutate(self, population):
#         population_next = []
#         for i in range(len(population)):
#             chromosome = population[i]
#             if random.random() < self.mutation_rate:
#                 mask = np.random.rand(len(chromosome)) < 0.05
#                 chromosome[mask] = False
#             population_next.append(chromosome)
#         return population_next

#     def generate(self, population):
#         # Selection, crossover and mutation
#         scores_sorted, population_sorted = self.fitness(population)
#         population = self.select(population_sorted)
#         population = self.crossover(population)
#         population = self.mutate(population)
#         # History
#         self.chromosomes_best.append(population_sorted[0])
#         self.scores_best.append(scores_sorted[0])
#         self.scores_avg.append(np.mean(scores_sorted))
        
#         return population

#     def fit(self, X, y):
 
#         self.chromosomes_best = []
#         self.scores_best, self.scores_avg  = [], []
        
#         self.dataset = X, y
#         self.n_features = X.shape[1]
        
#         population = self.initilize()
#         for i in range(self.n_gen):
#             population = self.generate(population)
            
#         return self 
    
#     @property
#     def support_(self):
#         return self.chromosomes_best[-1]

#     def plot_scores(self):
#         plt.plot(self.scores_best, label='Best')
#         plt.plot(self.scores_avg, label='Average')
#         plt.legend()
#         plt.ylabel('Scores')
#         plt.xlabel('Generation')
#         plt.show()

In [ ]:
# sel = GeneticSelector(estimator=RandomForestRegressor(), 
#                       n_gen=7, size=200, n_best=40, n_rand=40, 
#                       n_children=5, mutation_rate=0.05)
# sel.fit(x, y_log)

ValueError: ignored

In [ ]:
# sel.support_

In [ ]:
# from sklearn.feature_selection import VarianceThreshold

# select_VT = VarianceThreshold(0.00001)
# x_VT = select_VT.fit_transform(x)
# y_VT = select_VT.transform(y)
# x_VT.shape

In [ ]:
# model = RandomForestRegressor()
# model.fit(X_train_sel, y_train)
# print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
# print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

In [ ]:
# from sklearn.feature_selection import chi2, SelectKBest

# select_SKB = SelectKBest(chi2, k=10000)
# X_train_SKB = select_SKB.fit_transform(X_train)
# X_test_SKB = select_SKB.transform(X_test)

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 2.2MB/s 


In [ ]:
import lightgbm as lgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold, GroupKFold, TimeSeriesSplit, KFold

In [ ]:
ITERATIONS =20

# tunning
bayes_cv_tuner = BayesSearchCV(
    estimator = RandomForestRegressor(
        n_jobs=-1,
        verbose=0
    ),
    search_spaces = {
        'n_estimators': (30, 1000),
        'min_samples_split': (2, 100),
        'max_features': (0.1, 0.999, 'uniform'),
        'max_features': ['auto', 'sqrt'],
        'max_depth': (1, 20),
        'min_samples_leaf':(1, 20) 
    },    
    scoring = 'neg_mean_squared_log_error',
    cv = KFold(
        n_splits=5,
    ),
    n_jobs = -1,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 529
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
      
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
      
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    # all_models.to_csv(clf_name+"_cv_results.csv")

# Fit the model
result = bayes_cv_tuner.fit(X_train, y_train_log, callback=status_print)

Model #1
Best ROC-AUC: -0.0008
Best params: OrderedDict([('max_depth', 19), ('max_features', 'sqrt'), ('min_samples_leaf', 11), ('min_samples_split', 4), ('n_estimators', 227)])



KeyboardInterrupt: ignored

In [ ]:
pred_log = result.predict(X_test)
pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)

In [ ]:
import lightgbm as lgb

ITERATIONS = 9
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMRegressor(
        objective='regression',
        metric='rmsle',
        n_jobs=-1,
        verbose=0
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (2, 600),      
        'max_depth': (0, 60),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'subsample_freq': (0, 10),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'subsample_for_bin': (100000, 500000),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 100),
    },    
    scoring = 'neg_mean_squared_log_error',
    cv = TimeSeriesSplit(
        n_splits=4,
    ),
    n_jobs = -1,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
      
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
      
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    # all_models.to_csv(clf_name+"_cv_results.csv")

# Fit the model
lgbm = bayes_cv_tuner.fit(X_train, y_train_log, callback=status_print)

In [ ]:
pred_log = lgbm.predict(X_test)
pred_log_T = np.exp(1)**pred_log
mape(y_test, pred_log_T)